# ChildGPT-2_Bi
https://www.kaggle.com/baekseungyun/gpt-2-with-huggingface-pytorch<br>

In [ ]:
##Parameter setting
setEpoch = 10
setLearningRate = 0.0001
setEpsilon = 1e-8
setBatch = 32
setMaxLength = 256
setSeed = 42
labelNumber = 2
setTry = 31

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 101 kB 10.5 MB/s 
     |████████████████████████████████| 596 kB 52.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
for currentTry in range(1,setTry):
  
  # 1. Model and Tokenizer
  # https://github.com/SKT-AI/KoGPT2
  from transformers import set_seed, GPT2LMHeadModel, PreTrainedTokenizerFast, GPT2ForSequenceClassification, GPT2Config

  set_seed(731) # My Birthday!, you should get train_loss: 0.773, train_acc: 0.567 in epoch 0.

  model_config = GPT2Config.from_pretrained('skt/kogpt2-base-v2', num_labels=labelNumber) # Binary Classification
  model = GPT2ForSequenceClassification.from_pretrained('skt/kogpt2-base-v2', config=model_config)

  tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>') 
  tokenizer.padding_side = "left" # Very Important
  tokenizer.pad_token = tokenizer.eos_token

  print("Before patching: ",len(tokenizer.get_vocab()))

  # CHILDES데이터 사용하여 사전학습 모델 튜닝
  entireCorpus = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/CHILDES/CHILDES_all_spellchecked_corrected.txt"
  entireFr = open(entireCorpus, 'r')
  entireContents = entireFr.readlines()
  entireFr.close()

  import re

  entireSet = set()
  entireContentList = list()
  #공백을 기준으로 단어를 생성하고 추가하기
  for entireContent in entireContents:
      entireContent = re.sub('[^가-힣]', ' ', entireContent)
      entireContent = re.sub('[\s]+', ' ', entireContent)
      entireContentList.append(entireContent.replace("\n",""))

      entireContentSplit = entireContent.split(" ")
      for each in entireContentSplit:
        if each != "":
          entireSet.add(each)

  # print(len(entireSet)) #37898
  wordDic = {}
  for eachWord in entireSet:
    wordDic[eachWord] = 0

  for contentEach in entireContentList:
    contentEachSplit = contentEach.split(" ")
    for each in contentEachSplit:
      if each != "":
        wordDic[each] = wordDic[each] + 1

  wordDicSorted = dict(sorted(wordDic.items(), key=lambda x: x[1], reverse=True))

  #사전에 추가하기
  countNum = 1
  for key, value in wordDicSorted.items():
    if value > 1:
      countNum = countNum + 1
      tokenizer.add_tokens([key])

  # print(countNum) #3630   #10: 3630 #1: 17650 #2: 11848 #3: 9123 #4: 7435

  print("After patching: ",len(tokenizer.get_vocab()))

  model.resize_token_embeddings(len(tokenizer))
  model.config.pad_token_id = model.config.eos_token_id

  #2. Build Dataset
  import os
  import pandas as pd
  from torch.utils.data import Dataset

  class Dataset(Dataset):
      def __init__(self, train=True):
          super().__init__()
          self.train = train
          self.data = pd.read_csv(os.path.join("drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/", 'trainBi.csv' if train else 'testBi_GPT2_All.csv'))
      
      def __len__(self):
          return len(self.data)
      
      def __getitem__(self, index):
          record = self.data.iloc[index]
          text = record['Sentence']
          if self.train:
              return {'Sentence': text, 'label': record['Label']}
          else:
              return {'Sentence': text, 'label': '0'}

  train_dataset = Dataset(train=True)
  test_dataset = Dataset(train=False)
  #3. Data Collator
  class Gpt2ClassificationCollator(object):
      def __init__(self, tokenizer, max_seq_len=None):
          self.tokenizer = tokenizer
          self.max_seq_len = max_seq_len
          
          return
      
      def __call__(self, sequences):
          texts = [sequence['Sentence'] for sequence in sequences]
          labels = [int(sequence['label']) for sequence in sequences]
          inputs = self.tokenizer(text=texts,
                                  return_tensors='pt',
                                  padding=True,
                                  truncation=True,
                                  max_length=self.max_seq_len)
          inputs.update({'labels': torch.tensor(labels)})
          
          return inputs

  gpt2classificationcollator = Gpt2ClassificationCollator(tokenizer=tokenizer,
                                                          max_seq_len=setMaxLength)

  #4. DataLoader
  from torch.utils.data import DataLoader, random_split

  train_size = int(len(train_dataset) * 0.8)
  val_size = len(train_dataset) - train_size
  train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

  train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=setBatch,
                                shuffle=True,
                                collate_fn=gpt2classificationcollator)
  val_dataloader = DataLoader(dataset=val_dataset,
                              batch_size=setBatch,
                              shuffle=False,
                              collate_fn=gpt2classificationcollator)
  test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=setBatch,
                              shuffle=False,
                              collate_fn=gpt2classificationcollator)

  #5. Optimizer & Lr Scheduler
  from transformers import AdamW, get_cosine_schedule_with_warmup

  total_epochs = setEpoch

  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters,
                    lr=setLearningRate,
                    eps=setEpsilon)

  num_train_steps = len(train_dataloader) * total_epochs
  num_warmup_steps = int(num_train_steps * 0.1) 

  lr_scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps=num_warmup_steps,
                                                num_training_steps = num_train_steps)


  #6. Train & Validation
  import torch

  def train(dataloader, optimizer, scheduler, device_):
      global model
      model.train()
      
      prediction_labels = []
      true_labels = []
      
      total_loss = []
      
      for batch in dataloader:
          true_labels += batch['labels'].numpy().flatten().tolist()
          batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
          
          
          outputs = model(**batch)
          loss, logits = outputs[:2]
          logits = logits.detach().cpu().numpy()
          total_loss.append(loss.item())
          
          optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # prevent exploding gradient

          optimizer.step()
          scheduler.step()
          
          prediction_labels += logits.argmax(axis=-1).flatten().tolist()
      
      return true_labels, prediction_labels, total_loss

  def validation(dataloader, device_):
      global model
      model.eval()
      
      prediction_labels = []
      true_labels = []

      embedding_outputs = []
      
      total_loss = []

      outputs = []
      
      for batch in dataloader:
          true_labels += batch['labels'].numpy().flatten().tolist()
          batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}
          
          with torch.no_grad():
              outputs = model(**batch)
              loss, logits = outputs[:2]
              logits = logits.detach().cpu().numpy()
              total_loss.append(loss.item())

              prediction_labels += logits.argmax(axis=-1).flatten().tolist()

              embedding_outputs += logits.tolist()

              outputs = outputs
          
      return true_labels, prediction_labels, total_loss, outputs, embedding_outputs

  ##############################결과구문으로출력###################################
  def outreault(guess):
      guess = int(guess)
      outConstruction = ""
      if guess == 0:
          outConstruction = "agent-first"
      elif guess == 1:
          outConstruction = "theme-first"

      return outConstruction

  #7. Run

  outDir = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary_updated/OutputGPT2/gpt2Bi_All_T"+str(currentTry)+".csv"#"drive/My Drive/NLP/GH/BUCLD/Data/Binary/Output/gpt2Bi_BUCLD_SL256_LR000001.csv"
  f = open(outDir, 'w')
  f.write("epoch,sentence,originalLabel,predictedLabel,predictedConstruction,result"+"\n")

  from sklearn.metrics import classification_report, accuracy_score

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model.to(device)

  all_loss = {'train_loss': [], 'val_loss': []}
  all_acc = {'train_acc': [], 'val_acc': []}
  outputs = []

  for epoch in range(total_epochs):
      y, y_pred, train_loss = train(train_dataloader, optimizer, lr_scheduler, device)
      train_acc = accuracy_score(y, y_pred)
      
      y, y_pred, val_loss, outputs, logits_labels = validation(val_dataloader, device)
      val_acc = accuracy_score(y, y_pred)
      
      all_loss['train_loss'] += train_loss
      all_loss['val_loss'] += val_loss
      
      all_acc['train_acc'].append(train_acc)
      all_acc['val_acc'].append(val_acc)

      outputs = outputs

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch + 1, total_epochs))
      print('Training...')
      
      print(f'Epoch: {epoch}, train_loss: {torch.tensor(train_loss).mean():.3f}, train_acc: {train_acc:.3f}, val_loss: {torch.tensor(val_loss).mean():.3f}, val_acc: {val_acc:.3f}') 

      if (epoch+1) == 1 or (epoch+1) == 2 or (epoch+1) == 3 or (epoch+1) == 4 or (epoch+1) == 5 or (epoch+1) == 6 or (epoch+1) == 7 or (epoch+1) == 8 or (epoch+1) == 9 or (epoch+1) == 10 :#or (epoch+1) == 20 or (epoch+1) == 30:# or (epoch+1) == 40 or (epoch+1) == 50:
        # ###############################전체 모델 성능 평가########################################
        y, y_pred, val_loss, outputs, logits_labels = validation(test_dataloader, device)
        import pandas as pd

        testFileDir = fileDir = "drive/My Drive/NLP/GH/AgentThemeFirst/Data/Binary/testBi_All.csv"
        testFr = open(testFileDir, 'r')
        testContents = testFr.readlines()
        testFr.close()

        test = pd.DataFrame(columns=('Label1', 'Label2','Sentence'))
        i = 0
        for content in testContents:
            if i == 0:
                pass
            else:
                infos = content.split(",")
                label1 = int(infos[0])
                label2 = int(infos[1])
                sentence = infos[2].replace("\n", "")
                test.loc[i] = [label1, label2, sentence]
            i = i + 1

        test['Sentence'] = test['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
        test['Sentence'] = test['Sentence'].str.replace(r'\t+', " ", regex=True)
        test['Sentence'] = test['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

        # 리뷰 문장 추출
        testSentences = test['Sentence']

        totalNum = 0
        correctNum = 0
        for each in range(0, len(testSentences)):
            print(test['Label1'][each + 1], test['Label2'][each + 1], test['Sentence'][each + 1])
            print("y_pred", len(y_pred))
            guess = str(y_pred[each])
            if guess == str(test['Label1'][each + 1]) or guess == str(test['Label2'][each + 1]):
                print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(O)")
                f.write(str(epoch+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess)+ ",1" + "\n")
                correctNum = correctNum + 1
            else:
                f.write(str(epoch+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess) + ",0" + "\n")       
                print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(X)")
            totalNum = totalNum + 1

        print("totalNum: ", totalNum, " correctNum: ", correctNum, " accuracy: ", (correctNum/totalNum))

  f.close()

  print("")
  print("Training complete!")